In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())


In [ ]:
!hdfs dfs -ls /labs/laba01/ml-100k

In [ ]:
! hdfs dfs -head "/labs/laba01/ml-100k/u.data"

In [ ]:
#Решение через rdd
sc = spark.sparkContext
raw_data = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split("\t")).cache()
hist_all_raw = sorted(raw_data.\
                      map(lambda x: (x[2], 1)).\
                      reduceByKey(lambda a, b: a + b).\
                      collect())
hist_film_raw = sorted(raw_data.\
                       filter(lambda x: x[1] == "318").\
                       map(lambda x: (x[2], 1)).\
                       reduceByKey(lambda a, b: a + b).\
                       collect())
hist_all = [rating[1] for rating in hist_all_raw]
hist_film = [rating[1] for rating in hist_film_raw]

In [ ]:
#Решение через DF

from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, LongType

#Создадим схему - user id | item id | rating | timestamp.
schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("rating", IntegerType()),
    StructField("ts", LongType())
])

df = spark.read.csv("/labs/laba01/ml-100k/u.data", sep="\t", schema=schema)

hist_all_raw = sorted(df.groupBy('rating').agg({'item_id': 'count'}).collect())
hist_film_raw = sorted(df.where(df.item_id == 318).groupBy('rating').agg({'item_id': 'count'}).collect())

hist_all = [rating.asDict().get('count(item_id)') for rating in hist_all_raw]
hist_film = [rating.asDict().get('count(item_id)') for rating in hist_film_raw] 

In [ ]:
import json

result = {'hist_all': hist_all, 'hist_film':hist_film}

with open("lab01.json", 'w') as f:
    json.dump(result, f)

In [ ]:
with open("lab01.json", 'r') as f:
    print(f.read())

In [ ]:
spark.stop()